# Analyzing CSKG

This notebook performs various analyses on CSKG

Parameters are set up in the first cell so that we can run this notebook in batch mode.

### Parameters for invoking the notebook

- `cskg_path`: a folder containing the CSKG edges file and all the analysis products.
- `kg`: the name of the edge file.
- `delete_database`: whether to delete the SQL database before running the notebook: "" or "no" means don't delete it.

# Preamble

**Note** Download the CSKG KGTK graph from [here](https://drive.google.com/file/d/1mFwDJxav0H4CtA9UoPiwRHvPk_Zi4h0d/view?usp=sharing) and place it in the `../output` folder.

Set up paths and environment variables, make sure those varible are pointing to the correct file(path)

In [1]:
# Parameters
cskg_path = "../output" 
kg = "cskg_connected.kgtk.gz"
delete_database = "yes"

In [2]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

In [3]:
os.environ['CSKG'] = cskg_path
os.environ['KG'] = "{}/{}".format(cskg_path, kg)
os.environ['TMPKG'] = "{}/{}".format(cskg_path, "temp.cskg.normalize.1.kgtk.gz")
os.environ['NKG'] = "{}/{}".format(cskg_path, "cskg_normalized.tsv.gz")
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cskg_path)
os.environ['kypher'] = "kgtk query" # --graph-cache " + os.environ['STORE']
# os.environ['kypher'] = "time kgtk --debug query --graph-cache " + os.environ['STORE']

In [4]:
!echo $CSKG
!echo $KG
!echo $NKG
!echo $STORE
!echo $kypher

../output
../output/cskg.kgtk.tsv.gz
../output/cskg_normalized.tsv.gz
../output/wikidata.sqlite3.db
kgtk query


In [5]:
cd $cskg_path

/Users/filipilievski/mcs/cskg/output


In [6]:
if os.path.exists(delete_database) and delete_database != "no":
    print("Deleted database")
    !rm $STORE
else:
    print('No database exists')

No database exists


# Utilities

In [7]:
def bar_chart(data, x_column, y_column, title="", width=800):
    """Construct a simple bar chart with two properties"""
    bars = alt.Chart(data).mark_bar().encode(
        y=alt.Y(y_column, sort='-x'),
        x=x_column
    ).properties(
        title=title,
        width=width
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=x_column
    )

    return (bars + text)

In [8]:
import io
import pandas
import subprocess

def shell_df(command, shell=False, **kwargs):
    """
    Takes a shell command as a string and and reads the result into a Pandas DataFrame.
    
    Additional keyword arguments are passed through to pandas.read_csv.
    
    :param command: a shell command that returns tabular data
    :type command: str
    :param shell: passed to subprocess.Popen
    :type shell: bool
    
    :return: a pandas dataframe
    :rtype: :class:`pandas.dataframe`
    """
    proc = subprocess.Popen(command, 
                            shell=shell,
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE)
    output, error = proc.communicate()
    
    if proc.returncode == 0:
        if error:
            print(error.decode())
        with io.StringIO(output.decode()) as buffer:
            return pandas.read_csv(buffer, **kwargs)
    else:
        message = ("Shell command returned non-zero exit status: {0}\n\n"
                   "Command was:\n{1}\n\n"
                   "Standard error was:\n{2}")
        raise IOError(message.format(proc.returncode, command, error.decode()))

# Poking around

Print 10 edges to see what CSKG looks like

In [9]:
!kgtk query -i $KG \
--limit 10

id	node1	relation	node2	node1;label	node2;label	relation;label	relation;dimension	source	sentence
/c/en/0-/r/DefinedAs-/c/en/empty_set-0000	/c/en/0	/r/DefinedAs	/c/en/empty_set	"0"	"empty set"	"defined as"		"CN"	"[[0]] is the [[empty set]]."
/c/en/0-/r/DefinedAs-/c/en/first_limit_ordinal-0000	/c/en/0	/r/DefinedAs	/c/en/first_limit_ordinal	"0"	"first limit ordinal"	"defined as"		"CN"	"[[0]] is the [[first limit ordinal]]."
/c/en/0-/r/DefinedAs-/c/en/number_zero-0000	/c/en/0	/r/DefinedAs	/c/en/number_zero	"0"	"number zero"	"defined as"		"CN"	"[[0]] is the [[number zero]]."
/c/en/0-/r/HasContext-/c/en/internet_slang-0000	/c/en/0	/r/HasContext	/c/en/internet_slang	"0"	"internet slang"	"has context"		"CN"	
/c/en/0-/r/HasProperty-/c/en/pronounced_zero-0000	/c/en/0	/r/HasProperty	/c/en/pronounced_zero	"0"	"pronounced zero"	"has property"		"CN"	"[[\"0\"]] is [[pronounced zero]]"
/c/en/0-/r/IsA-/c/en/set_containing_one_element-0000	/c/en/0	/r/IsA	/c/en/set_containing_one_element	"0"	"set contai

Normalize the file so that it is easier to process with Kypher

In [11]:
!kgtk normalize --verbose -i $KG -o $TMPKG --columns-to-lower 'relation;dimension' source sentence 'node1;label' 'relation;label' 'node2;label'

Opening the input file: ../output/cskg.kgtk.tsv.gz
KgtkReader: File_path.suffix: .gz
KgtkReader: reading gzip ../output/cskg.kgtk.tsv.gz
header: id	node1	relation	node2	node1;label	node2;label	relation;label	relation;dimension	source	sentence
input format: kgtk
KgtkReader: Special columns: node1=1 label=2 node2=3 id=0
KgtkReader: Reading an edge file.
Node1 column name: node1
Label column name: relation
Node2 column name: node2
Id column name: id
The following columns will be lowered or normalized
 node1;label from node1 (label 'label')
 node2;label from node2 (label 'label')
 relation;label from relation (label 'label')
 relation;dimension from relation (label 'dimension')
 source from id (label 'source')
 sentence from id (label 'sentence')
The output columns are: id node1 relation node2
Opening the output file: ../output/temp.cskg.normalize.1.kgtk.gz
File_path.suffix: .gz
KgtkWriter: writing gzip ../output/temp.cskg.normalize.1.kgtk.gz
header: id	node1	relation	node2
Read 6001531 ro

In [12]:
!kgtk query -i $TMPKG \
--limit 10

id	node1	relation	node2
/c/en/0-/r/DefinedAs-/c/en/empty_set-0000	/c/en/0	/r/DefinedAs	/c/en/empty_set
/c/en/0-/r/DefinedAs-/c/en/first_limit_ordinal-0000	/c/en/0	/r/DefinedAs	/c/en/first_limit_ordinal
/c/en/0-/r/DefinedAs-/c/en/number_zero-0000	/c/en/0	/r/DefinedAs	/c/en/number_zero
/c/en/0-/r/HasContext-/c/en/internet_slang-0000	/c/en/0	/r/HasContext	/c/en/internet_slang
/c/en/0-/r/HasProperty-/c/en/pronounced_zero-0000	/c/en/0	/r/HasProperty	/c/en/pronounced_zero
/c/en/0-/r/IsA-/c/en/set_containing_one_element-0000	/c/en/0	/r/IsA	/c/en/set_containing_one_element
/c/en/0-/r/RelatedTo-/c/en/1-0000	/c/en/0	/r/RelatedTo	/c/en/1
/c/en/0-/r/RelatedTo-/c/en/2-0000	/c/en/0	/r/RelatedTo	/c/en/2
/c/en/0.22_inch_calibre-/r/IsA-/c/en/5.6_millimetres-0000	/c/en/0.22_inch_calibre	/r/IsA	/c/en/5.6_millimetres
/c/en/0/a/wn-/r/SimilarTo-/c/en/cardinal/a/wn-0000	/c/en/0/a/wn	/r/SimilarTo	/c/en/cardinal/a/wn


Rename the columns to the standard `node1/label/node2` and add ids

In [13]:
!kgtk rename-columns --mode NONE -i $TMPKG --output-columns id node1 label node2 \
/ add-id --id-style node1-label-node2 -o $NKG

Count the number of edges and nodes

In [14]:
!kgtk query -i $NKG \
--match '(n1)-[e]->(n2)' \
--return 'count(e) as num_edges, count(distinct n1) as num_nodes, count(distinct e.label) as num_relations, count(distinct n2) as num_values' \
| column -t -s $'\t' 

num_edges  num_nodes  num_relations  num_values
15116999   8162557    61             3284811


# Some Statistics

In [15]:
command = "$kypher -i $NKG \
--match '(n1)-[e]->(n2)' \
--return 'distinct e.label, count(distinct n1) as nodes' \
--order-by 'count(distinct n1) desc'"
stats = shell_df(command, shell=True, sep='\t')

In [16]:
bar_chart(stats[:20], 'nodes', 'label', title="Relations in CSKG")

alt.LayerChart(...)

### Distribution of edges in each data source

In [17]:
command = "$kypher -i $NKG \
--match '(n1)-[r]->(n2), (r)-[l:source]->(s)' \
--return 's as source, count(distinct r) as `count of relations`' \
--order-by 'count(distinct r) desc'"
data = shell_df(command, shell=True, sep='\t')
bar_chart(data, 'count of relations', 'source', title="CSKG Relation Counts")

alt.LayerChart(...)

Compute the distribuiton of relations in each data source

In [18]:
command = "$kypher -i $NKG \
--match '(n1)-[r {label: label}]->(n2), (r)-[:source]->(source:`\"SOURCE\"`)' \
--return 'label as relation, count(distinct n1) as count' \
--order-by 'count(distinct n1) desc'"
datasets = []
for source in ["CN", "WN", "AT", "VG", "FN", "WD", "RG"]:
    data = shell_df(command.replace("SOURCE", source), shell=True, sep='\t')
    datasets.append(data)

In [19]:
bar_chart(datasets[0], 'count', 'relation', title="ConceptNet: Count Of Relations", width=200)

alt.LayerChart(...)

In [20]:
bar_chart(datasets[1], 'count', 'relation', title="WordNet: Count Of Relations", width=200)

alt.LayerChart(...)

In [21]:
bar_chart(datasets[2], 'count', 'relation', title="Atomic: Count Of Relations", width=200)

alt.LayerChart(...)

In [22]:
bar_chart(datasets[3], 'count', 'relation', title="Visual Genome: Count Of Relations", width=200)

alt.LayerChart(...)

In [23]:
bar_chart(datasets[4], 'count', 'relation', title= "FrameNet: Count Of Relations", width=200)

alt.LayerChart(...)

In [24]:
bar_chart(datasets[5], 'count', 'relation', "Wikidata: Count Of Relations", width=200)

alt.LayerChart(...)

In [25]:
bar_chart(datasets[6], 'count', 'relation', title="Roget: Count Of Relations", width=200)

alt.LayerChart(...)

### ConceptNet nodes that contain `catch` and `throw`

In [26]:
command = "kgtk query -i $NKG \
--match '(n1)-[r {label: label}]->(n2), (r)-[:source]->(source)' \
--where 'source = $s and n1 =~ \".*/catch/.*\"' \
--return 'distinct n1 as node1' \
--spara s='CN' \
--limit 100"

catch_df = shell_df(command, shell=True, sep='\t')

for i, row in catch_df.iterrows():
    print("https://www.conceptnet.io"+row['node1'])

https://www.conceptnet.io/c/en/catch/n
https://www.conceptnet.io/c/en/catch/n/wn/act
https://www.conceptnet.io/c/en/catch/n/wn/artifact
https://www.conceptnet.io/c/en/catch/n/wn/attribute
https://www.conceptnet.io/c/en/catch/n/wn/object
https://www.conceptnet.io/c/en/catch/n/wn/person
https://www.conceptnet.io/c/en/catch/n/wn/quantity
https://www.conceptnet.io/c/en/catch/v
https://www.conceptnet.io/c/en/catch/v/wn/body
https://www.conceptnet.io/c/en/catch/v/wn/competition
https://www.conceptnet.io/c/en/catch/v/wn/emotion
https://www.conceptnet.io/c/en/catch/v/wn/motion
https://www.conceptnet.io/c/en/catch/v/wn/perception
https://www.conceptnet.io/c/en/catch/v/wn/possession
https://www.conceptnet.io/c/en/catch/v/wn/social


In [27]:
command = "kgtk query -i $NKG \
--match '(n1)-[r {label: label}]->(n2), (r)-[:source]->(source)' \
--where 'source in [$cn] and n1 =~ \".*/throw/.*\"' \
--return 'distinct n1 as node1' \
--spara cn='CN' --spara wd='WD' --spara vg='VG' \
 --limit 100"

throw_df = shell_df(command, shell=True, sep='\t')

for i, row in throw_df.iterrows():
    print("https://www.conceptnet.io"+row['node1'])

https://www.conceptnet.io/c/en/throw/n
https://www.conceptnet.io/c/en/throw/n/wikt/en_1
https://www.conceptnet.io/c/en/throw/n/wikt/en_2
https://www.conceptnet.io/c/en/throw/n/wikt/en_3
https://www.conceptnet.io/c/en/throw/n/wikt/en_4
https://www.conceptnet.io/c/en/throw/n/wn/artifact
https://www.conceptnet.io/c/en/throw/n/wn/event
https://www.conceptnet.io/c/en/throw/n/wn/state
https://www.conceptnet.io/c/en/throw/n/wp/grappling
https://www.conceptnet.io/c/en/throw/v
https://www.conceptnet.io/c/en/throw/v/wikt/en_1
https://www.conceptnet.io/c/en/throw/v/wikt/en_2
https://www.conceptnet.io/c/en/throw/v/wn/cognition
https://www.conceptnet.io/c/en/throw/v/wn/communication
https://www.conceptnet.io/c/en/throw/v/wn/emotion


### ConceptNet nodes that contain `dog` and `frisbee`

In [28]:
command = "kgtk query -i $NKG \
--match '(n1)-[r {label: label}]->(n2), (r)-[:source]->(source)' \
--where 'source in [$cn] and n1 =~ \".*/dog/.*\"' \
--return 'distinct n1 as node1' \
--spara cn='CN' \
--limit 100"

dog_df = shell_df(command, shell=True, sep='\t')

for i, row in dog_df.iterrows():
    print("https://www.conceptnet.io"+row['node1'])

https://www.conceptnet.io/c/en/dog/n
https://www.conceptnet.io/c/en/dog/n/wn/animal
https://www.conceptnet.io/c/en/dog/n/wn/artifact
https://www.conceptnet.io/c/en/dog/n/wn/food
https://www.conceptnet.io/c/en/dog/n/wn/person
https://www.conceptnet.io/c/en/dog/n/wp/goya
https://www.conceptnet.io/c/en/dog/v
https://www.conceptnet.io/c/en/dog/v/wn/motion


In [31]:
command = "kgtk query -i $NKG \
--match '(n1)-[r {label: label}]->(n2), (r)-[:source]->(source)' \
--where 'source in [$cn] and n1 =~ \".*/frisbee/.*\"' \
--return 'distinct n1 as node1' \
--spara cn='CN' --spara wd='WD' --spara vg='VG' \
--limit 100"

frisbee_df = shell_df(command, shell=True, sep='\t')

for i, row in frisbee_df.iterrows():
    print("https://www.conceptnet.io"+row['node1'])

https://www.conceptnet.io/c/en/frisbee/n
https://www.conceptnet.io/c/en/frisbee/v


### Get all the nodes in ConceptNet, Wikidata-CS and VisualGenone that contain `frisbee`

In [32]:
!kgtk query -i $NKG \
--match '(n1)-[r {label: label}]->(n2), (r)-[:source]->(source), (n1)-[:label]->(n1_label)' \
--where 'source in [$cn, $vg, $wd] and n1 =~ ".*frisbees?.*"' \
--return 'distinct source as source, n1 as node1, n1_label as `node1 label`, label as relation, n2 as node2' \
--order-by 'source, n1' \
--spara cn='CN' --spara wd='WD' --spara vg='VG'\
-o $CSKG/frisbee.tsv

We have many edges that relate to `frisbee`

In [33]:
!wc -l $CSKG/frisbee.tsv

   12828 ../output/frisbee.tsv


In [34]:
!head $CSKG/frisbee.tsv | column -t -s $'\t' 

source  node1                         node1 label               relation            node2
"CN"    /c/en/capturing_frisbee       "capturing frisbee"       /r/HasPrerequisite  /c/en/coordination
"CN"    /c/en/dogs_catching_frisbees  "dogs catching frisbees"  /r/AtLocation       /c/en/park
"CN"    /c/en/frisbee                 "frisbee"                 /r/AtLocation       /c/en/air
"CN"    /c/en/frisbee                 "frisbee"                 /r/AtLocation       /c/en/deadhead's_van
"CN"    /c/en/frisbee                 "frisbee"                 /r/AtLocation       /c/en/frisbee_golf_course
"CN"    /c/en/frisbee                 "frisbee"                 /r/AtLocation       /c/en/park
"CN"    /c/en/frisbee                 "frisbee"                 /r/AtLocation       /c/en/roof
"CN"    /c/en/frisbee                 "frisbee"                 /r/AtLocation       /c/en/toy_chest
"CN"    /c/en/frisbee                 "frisbee"                 /r/AtLocation       /c/en/tree


In [35]:
!tail $CSKG/frisbee.tsv | column -t -s $'\t' 

"VG"  wn:frisbee.n.01  "gray frisbee"          mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "frisbee in hand"       mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "plates"                mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "frisbee in the air"    mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "frisbee is purple"     mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "purple frisbee"        mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "frisbee is round"      mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "plastic"               mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "frisbee is in hand"    mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
"VG"  wn:frisbee.n.01  "white frisbee in the"  mw:MayHaveProperty  /c/en/yellow/n/wn/attribute
